tinjano@github, Feb 2024

This notebook is part of the repository at https://github.com/tinjano/ad1.

This notebook collects data about Google advertising using BigQuery. The data is found in `creative_stats` and `removed_creative_stats` (data about ads that were removed by Google) tables, part of the `google_ads_transparency_ceneter` dataset of the `bigquery-public-data` project. In particular, we collect all data about advertisements that were shown in Croatia with all data already available (for some entries, data will become available at a set date in the future; see schemes below).

To run this or a similar notebook, you will need to have the credentials for a Google Cloud service account with adequate permissions. Be careful about potentially processing very large amount of data.

# Imports

In [1]:
# Python standard utilities
import os
os.makedirs('data/', exist_ok=True)
from pprint import pprint as pretty_print
from datetime import datetime

# data processing
import pandas as pd

# google
from google.cloud import bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'google-credentials.json'

# auxiliary modules
from aux import country_to_abbrev, abbrev_to_country

# config
project = 'bigquery-public-data'
dataset = 'google_ads_transparency_center'

# Tables and Schemes

In [2]:
client = bigquery.Client()

dataset_ref = client.dataset(dataset, project=project)
dataset_ = client.get_dataset(dataset_ref)

for table_ in list(client.list_tables(dataset_)):
    id_ = table_.table_id
    if '-temp-' in id_:
        continue
        
    print(f'\x1b[31mTable id: {table_.table_id}\x1b[0m')
    table = client.get_table('.'.join([project, dataset, table_.table_id]))
    pretty_print(vars(table).get('_properties').get('schema'))
    print('\n')

Table id: creative_stats
{'fields': [{'description': 'A unique identifier for a specific advertiser',
             'mode': 'NULLABLE',
             'name': 'advertiser_id',
             'type': 'STRING'},
            {'description': 'A unique identifier for a specific ad',
             'mode': 'NULLABLE',
             'name': 'creative_id',
             'type': 'STRING'},
            {'description': 'A link to the relevant Ads Transparency Center '
                            'page for this ad',
             'mode': 'NULLABLE',
             'name': 'creative_page_url',
             'type': 'STRING'},
            {'description': 'A description of the ad type',
             'mode': 'NULLABLE',
             'name': 'ad_format_type',
             'type': 'STRING'},
            {'description': "The advertiser's name for transparency reporting "
                            'and disclosures',
             'mode': 'NULLABLE',
             'name': 'advertiser_disclosed_name',
             'type

# `creative_stats` table

## Query

In [4]:
table = 'creative_stats'
print('.'.join([project, dataset, table]))

bigquery-public-data.google_ads_transparency_center.creative_stats


In [6]:
query_job = client.query(
    f'''
    SELECT
    creative_page_url,
    ad_format_type,
    advertiser_disclosed_name,
    advertiser_location,
    advertiser_verification_status,
    
    PARSE_TIMESTAMP('%Y-%m-%d', first_shown) AS first_shown,
    PARSE_TIMESTAMP('%Y-%m-%d', last_shown) AS last_shown,
    PARSE_TIMESTAMP('%Y-%m-%d', times_shown_end_date) AS  times_shown_end_date,
    
    times_shown_lower_bound AS times_shown_lower,
    times_shown_upper_bound AS times_shown_upper,
    
    IF(audience_selection_approach_info.demographic_info = 'CRITERIA_UNUSED',  0,  1) AS selection_demographic,
    IF(audience_selection_approach_info.geo_location = 'CRITERIA_UNUSED',  0,  1) AS selection_geo,
    IF(audience_selection_approach_info.contextual_signals = 'CRITERIA_UNUSED',  0,  1) AS selection_contextual,
    IF(audience_selection_approach_info.customer_lists = 'CRITERIA_UNUSED',  0,  1) AS selection_customer_lists,
    IF(audience_selection_approach_info.topics_of_interest = 'CRITERIA_UNUSED',  0,  1) AS selection_toi,

    topic
    
    
    FROM `{'.'.join([project, dataset, table])}`,
        UNNEST(region_stats)

    WHERE
    region_code='HR' AND
    times_shown_availability_date IS NULL
    '''
)

In [7]:
print('Starting time: ', datetime.now())
%time df = query_job.to_dataframe()

Starting time:  2024-02-15 07:25:34.950907
CPU times: user 1min 4s, sys: 2.98 s, total: 1min 7s
Wall time: 15min 52s


## Cleaning

In [8]:
df.advertiser_location = df.advertiser_location.apply(lambda x: abbrev_to_country.get(x.strip('\"')))

In [9]:
for col in ['ad_format_type', 'advertiser_verification_status', 'topic']:
    df.loc[:, col] = df.loc[:, col].str.lower()

In [10]:
df['days_running'] = (df.last_shown - df.first_shown).dt.days

In [11]:
df = df.astype({
    'ad_format_type': 'category',
    'advertiser_location': 'category',
    'advertiser_verification_status': 'category',
    'selection_demographic': 'boolean',
    'selection_geo': 'boolean',
    'selection_contextual': 'boolean',
    'selection_customer_lists': 'boolean',
    'selection_toi': 'boolean',
    'topic': 'category'
})

In [12]:
df.shape, df.dtypes

((3154151, 17),
 creative_page_url                              object
 ad_format_type                               category
 advertiser_disclosed_name                      object
 advertiser_location                          category
 advertiser_verification_status               category
 first_shown                       datetime64[us, UTC]
 last_shown                        datetime64[us, UTC]
 times_shown_end_date              datetime64[us, UTC]
 times_shown_lower                               Int64
 times_shown_upper                               Int64
 selection_demographic                         boolean
 selection_geo                                 boolean
 selection_contextual                          boolean
 selection_customer_lists                      boolean
 selection_toi                                 boolean
 topic                                        category
 days_running                                    int64
 dtype: object)

In [13]:
df

,creative_page_url,ad_format_type,advertiser_disclosed_name,advertiser_location,advertiser_verification_status,first_shown,last_shown,times_shown_end_date,times_shown_lower,times_shown_upper,selection_demographic,selection_geo,selection_contextual,selection_customer_lists,selection_toi,topic,days_running
0,https://adstransparency.google.com/advertiser/...,text,Zauchensee Walchhofer GmbH,Austria,unverified,2023-03-01 00:00:00+00:00,2024-02-13 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,travel & tourism,349
1,https://adstransparency.google.com/advertiser/...,text,ALLESkralle GmbH,Austria,verified,2023-03-01 00:00:00+00:00,2023-10-28 00:00:00+00:00,2023-10-28 00:00:00+00:00,0,1000,True,True,True,False,False,jobs & education,241
2,https://adstransparency.google.com/advertiser/...,text,AutoScout24 AS GmbH,Austria,verified,2023-03-01 00:00:00+00:00,2023-12-03 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,True,False,autos & vehicles,277
3,https://adstransparency.google.com/advertiser/...,text,Donau-Universität Krems,Austria,verified,2023-03-20 00:00:00+00:00,2024-01-02 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,jobs & education,288
4,https://adstransparency.google.com/advertiser/...,text,Harry's Home Holding AG,Austria,verified,2023-09-06 00:00:00+00:00,2024-02-13 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,travel & tourism,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154146,https://adstransparency.google.com/advertiser/...,text,wagner andres alvarez tejeda,United States,verified,2023-07-04 00:00:00+00:00,2024-02-07 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,travel & tourism,218
3154147,https://adstransparency.google.com/advertiser/...,video,ABI GLOBAL LTD,Virgin Islands (UK),verified,2023-03-01 00:00:00+00:00,2023-09-15 00:00:00+00:00,2023-09-15 00:00:00+00:00,1000,2000,True,True,True,False,False,"hobbies, games & leisure",198
3154148,https://adstransparency.google.com/advertiser/...,video,INFINITY TECHNOLOGIES GLOBAL LTD,Virgin Islands (UK),verified,2023-07-28 00:00:00+00:00,2023-08-06 00:00:00+00:00,2023-08-06 00:00:00+00:00,0,1000,True,False,True,False,False,"hobbies, games & leisure",9
3154149,https://adstransparency.google.com/advertiser/...,video,VTN GLOBAL LTD,Virgin Islands (UK),verified,2023-11-15 00:00:00+00:00,2023-11-26 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,family & community,11


## Save to disk

In [14]:
df.to_parquet('data/google-ads-hr-2.parquet')

# `removed_creative_stats`

## Query

In [16]:
table = 'removed_creative_stats'
print('.'.join([project, dataset, table]))

bigquery-public-data.google_ads_transparency_center.removed_creative_stats


In [17]:
query_job = client.query(
    f'''
    SELECT
    creative_page_url,
    PARSE_TIMESTAMP('%Y-%m-%d', first_shown) AS first_shown,
    PARSE_TIMESTAMP('%Y-%m-%d', last_shown) AS last_shown,
    PARSE_TIMESTAMP('%Y-%m-%d', times_shown_end_date) AS  times_shown_end_date,
    times_shown_lower_bound AS times_shown_lower,
    times_shown_upper_bound AS times_shown_upper,

    IF(audience_selection_approach_info.demographic_info = 'CRITERIA_UNUSED',  0,  1) AS selection_demographic,
    IF(audience_selection_approach_info.geo_location = 'CRITERIA_UNUSED',  0,  1) AS selection_geo,
    IF(audience_selection_approach_info.contextual_signals = 'CRITERIA_UNUSED',  0,  1) AS selection_contextual,
    IF(audience_selection_approach_info.customer_lists = 'CRITERIA_UNUSED',  0,  1) AS selection_customer_lists,
    IF(audience_selection_approach_info.topics_of_interest = 'CRITERIA_UNUSED',  0,  1) AS selection_toi,

    disapproval[0].removal_reason AS removal_reason,
    disapproval[0].violation_category AS violation_category,
    disapproval[0].use_of_automated_means AS automated,
    disapproval[0].decision_type AS decision_type,


    FROM `{'.'.join([project, dataset, table])}`,
        UNNEST(region_stats)

    WHERE
    region_code='HR' AND
    times_shown_availability_date IS NULL
    '''
)

In [18]:
print('Starting time: ', datetime.now())
%time df_removed = query_job.to_dataframe()

Starting time:  2024-02-15 07:41:47.504215
CPU times: user 1.52 s, sys: 145 ms, total: 1.67 s
Wall time: 19 s


## Cleaning

In [19]:
df_removed.removal_reason = df_removed.removal_reason.str.lower()

In [20]:
df_removed.violation_category = (df_removed.violation_category
                                 .str.split('_')
                                 .apply(lambda x: list(map(str.lower, x)))
                                 .str.join(' '))

In [21]:
df_removed['days_running'] = (df_removed.last_shown - df_removed.first_shown).dt.days

In [22]:
df_removed = df_removed.astype({
    'selection_demographic': 'boolean',
    'selection_geo': 'boolean',
    'selection_contextual': 'boolean',
    'selection_customer_lists': 'boolean',
    'selection_toi': 'boolean',

    'removal_reason': 'category',
    'violation_category': 'category',
    'automated': 'boolean',
    'decision_type': 'category'
})

In [23]:
df_removed.shape, df_removed.dtypes

((86363, 16),
 creative_page_url                        object
 first_shown                 datetime64[us, UTC]
 last_shown                  datetime64[us, UTC]
 times_shown_end_date        datetime64[us, UTC]
 times_shown_lower                         Int64
 times_shown_upper                         Int64
 selection_demographic                   boolean
 selection_geo                           boolean
 selection_contextual                    boolean
 selection_customer_lists                boolean
 selection_toi                           boolean
 removal_reason                         category
 violation_category                     category
 automated                               boolean
 decision_type                          category
 days_running                              int64
 dtype: object)

In [24]:
df_removed

,creative_page_url,first_shown,last_shown,times_shown_end_date,times_shown_lower,times_shown_upper,selection_demographic,selection_geo,selection_contextual,selection_customer_lists,selection_toi,removal_reason,violation_category,automated,decision_type,days_running
0,https://adstransparency.google.com/removed/cre...,2023-09-01 00:00:00+00:00,2023-10-24 00:00:00+00:00,2023-10-24 00:00:00+00:00,0,1000,True,True,True,False,True,editorial,editorial and technical,False,Google investigation,53
1,https://adstransparency.google.com/removed/cre...,2023-09-16 00:00:00+00:00,2023-09-20 00:00:00+00:00,2023-09-20 00:00:00+00:00,9000,10000,True,True,True,True,False,editorial,editorial and technical,False,Google investigation,4
2,https://adstransparency.google.com/removed/cre...,2023-11-08 00:00:00+00:00,2023-12-03 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,True,misrepresentation,prohibited practices,False,Google investigation,25
3,https://adstransparency.google.com/removed/cre...,2023-03-23 00:00:00+00:00,2024-01-23 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,True,True,True,False,False,healthcare and medicines,restricted content and features,True,Google investigation,306
4,https://adstransparency.google.com/removed/cre...,2023-08-09 00:00:00+00:00,2023-09-03 00:00:00+00:00,2023-09-03 00:00:00+00:00,0,1000,True,True,True,False,False,account suspension,prohibited practices,False,Google investigation,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86358,https://adstransparency.google.com/removed/cre...,2023-08-15 00:00:00+00:00,2023-11-30 00:00:00+00:00,2023-11-16 00:00:00+00:00,0,1000,False,True,True,False,False,account suspension,prohibited practices,False,Google investigation,107
86359,https://adstransparency.google.com/removed/cre...,2023-06-13 00:00:00+00:00,2023-08-13 00:00:00+00:00,2023-08-13 00:00:00+00:00,50000,60000,False,True,True,False,False,editorial,editorial and technical,False,Google investigation,61
86360,https://adstransparency.google.com/removed/cre...,2023-08-30 00:00:00+00:00,2023-11-05 00:00:00+00:00,2023-11-05 00:00:00+00:00,0,1000,False,True,True,False,False,account suspension,prohibited practices,False,Google investigation,67
86361,https://adstransparency.google.com/removed/cre...,2023-10-03 00:00:00+00:00,2023-10-03 00:00:00+00:00,2023-10-03 00:00:00+00:00,0,1000,False,True,True,True,False,trademarks,restricted content and features,False,Google investigation,0


## Save to disk

In [25]:
df_removed.to_parquet('data/google-ads-hr-removed-2.parquet')

# Problems in the data
Note that the `times_shown_upper` column (`times_shown_upper_bound` in the original tables) contains instances of an extremely large number (2^64 - 1), either as the result of an error or as a way to signal that there is no upper bound.

In [28]:
df.times_shown_upper.value_counts()

times_shown_upper
1000                   2756478
2000                    129188
3000                     54392
4000                     31782
15000                    22619
5000                     21307
6000                     15682
20000                    13236
7000                     12154
8000                      9616
25000                     8843
9000                      8002
30000                     6674
10000                     6623
35000                     4988
60000                     4577
40000                     4136
125000                    3852
70000                     3610
45000                     3373
150000                    2798
80000                     2788
50000                     2751
90000                     2344
175000                    2152
100000                    2061
300000                    1842
200000                    1679
225000                    1406
350000                    1340
250000                    1161
600000               

In [29]:
df_removed.times_shown_upper.value_counts()

times_shown_upper
1000                   74537
2000                    3494
3000                    1623
4000                     868
15000                    690
5000                     587
20000                    452
6000                     378
7000                     339
25000                    331
30000                    271
8000                     268
10000                    208
9000                     196
35000                    187
60000                    150
125000                   142
40000                    139
45000                    125
70000                    118
50000                    104
80000                     85
175000                    83
150000                    81
300000                    76
90000                     75
100000                    72
350000                    69
225000                    63
200000                    57
600000                    54
250000                    54
400000                    48
500000                   